In [2]:
!pip install faker

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 822.6 kB/s eta 0:00:02
   --------------------- ------------------ 1.0/1.9 MB 1.2 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/1.9 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 1.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import random
from faker import Faker
from datetime import timedelta
import numpy as np

fake = Faker()
random.seed(42)
np.random.seed(42)

num_tickets = 5000
categories = ['Billing', 'Fraud', 'Card Reissue', 'Travel Booking', 'Dispute']

data = []

for _ in range(num_tickets):
    created_at = fake.date_between(start_date='-1y', end_date='-10d')
    resolution_days = random.randint(1, 15)
    resolved_at = created_at + timedelta(days=resolution_days)
    
    entry = {
        'ticket_id': fake.uuid4(),
        'customer_id': fake.uuid4(),
        'agent_id': f"AGENT{random.randint(100, 150)}",
        'created_at': created_at,
        'resolved_at': resolved_at,
        'resolution_days': resolution_days,
        'category': random.choice(categories),
        'satisfaction_score': random.randint(1, 10),
    }
    data.append(entry)

df_perf = pd.DataFrame(data)


In [4]:
# Define SLA breach (if resolution > 7 days)
df_perf['sla_breached'] = df_perf['resolution_days'] > 7

# Incentive eligible if:
# - resolution_days <= 7
# - satisfaction_score >= 8
df_perf['incentive_eligible'] = df_perf.apply(
    lambda row: row['resolution_days'] <= 7 and row['satisfaction_score'] >= 8, axis=1
)

print("Sample rows:")
print(df_perf.head())


Sample rows:
                              ticket_id                           customer_id  \
0  a96eb1bb-c325-4fce-996f-4fd4ccd65811  8af38ba8-2519-476a-a3db-61fb81b7b9c8   
1  af831c37-a0d4-4e17-b17c-ee0c367ddc0c  2aa6e1b6-8a2d-467f-926f-a1dd625713ec   
2  8f3bd3f0-e713-483d-9f12-f38d06030d94  9a69720e-a961-47a6-810b-2b71e4b38ecd   
3  9e0aaada-20df-4ee9-936e-7da91549cbb9  07933791-0671-487c-bfb5-a8932e735c5f   
4  4a1b5739-66cc-424a-8d92-719a2fce11c9  3292460f-00e9-4691-b38c-7c1e514e7b86   

   agent_id  created_at resolved_at  resolution_days category  \
0  AGENT107  2025-04-27  2025-05-08               11  Billing   
1  AGENT114  2024-08-10  2024-08-14                4    Fraud   
2  AGENT147  2025-05-30  2025-06-10               11  Dispute   
3  AGENT127  2024-11-13  2024-11-23               10  Billing   
4  AGENT113  2024-10-31  2024-11-02                2    Fraud   

   satisfaction_score  sla_breached  incentive_eligible  
0                   5          True               F

In [5]:
df_perf.to_csv("../data/agent_performance.csv", index=False)
print("Saved: ../data/agent_performance.csv")


Saved: ../data/agent_performance.csv


In [6]:
import pandas as pd

# Load data
df_perf = pd.read_csv("../data/agent_performance.csv")

# Convert dates
df_perf['created_at'] = pd.to_datetime(df_perf['created_at'])
df_perf['resolved_at'] = pd.to_datetime(df_perf['resolved_at'])

# Agent-level summary
agent_kpis = df_perf.groupby('agent_id').agg(
    total_tickets=('ticket_id', 'count'),
    avg_resolution_days=('resolution_days', 'mean'),
    avg_satisfaction=('satisfaction_score', 'mean'),
    sla_breaches=('sla_breached', 'sum'),
    incentive_eligible_count=('incentive_eligible', 'sum')
).reset_index()

# Compute incentive rate
agent_kpis['incentive_rate'] = (agent_kpis['incentive_eligible_count'] / agent_kpis['total_tickets']).round(2)

print("Top 5 agent performance metrics:")
print(agent_kpis.head())


Top 5 agent performance metrics:
   agent_id  total_tickets  avg_resolution_days  avg_satisfaction  \
0  AGENT100             86             8.581395          5.627907   
1  AGENT101             89             8.471910          5.460674   
2  AGENT102            105             7.857143          5.742857   
3  AGENT103             86             7.720930          5.534884   
4  AGENT104             90             7.588889          5.533333   

   sla_breaches  incentive_eligible_count  incentive_rate  
0            49                        10            0.12  
1            54                        12            0.13  
2            54                        17            0.16  
3            45                        13            0.15  
4            47                        13            0.14  


In [7]:
category_summary = df_perf.groupby('category').agg(
    total_tickets=('ticket_id', 'count'),
    avg_resolution_days=('resolution_days', 'mean'),
    sla_breach_rate=('sla_breached', lambda x: round(x.mean(), 2)),
    avg_satisfaction=('satisfaction_score', 'mean')
).reset_index()

print("Issue category performance summary:")
print(category_summary)


Issue category performance summary:
         category  total_tickets  avg_resolution_days  sla_breach_rate  \
0         Billing           1028             8.013619             0.53   
1    Card Reissue            992             8.024194             0.54   
2         Dispute            973             8.133607             0.54   
3           Fraud            985             7.899492             0.52   
4  Travel Booking           1022             8.045988             0.54   

   avg_satisfaction  
0          5.431907  
1          5.666331  
2          5.565262  
3          5.323858  
4          5.490215  


In [8]:
agent_kpis.to_csv("../outputs/agent_performance_kpis.csv", index=False)
category_summary.to_csv("../outputs/category_performance_summary.csv", index=False)

print("Exported: agent_performance_kpis.csv and category_performance_summary.csv")


Exported: agent_performance_kpis.csv and category_performance_summary.csv
